# 3D wavelet and UMAP # 

In [1]:
import cv2
import numpy as np
import os
import pywt
from scipy import sparse


def load_video_frames(folder_path):
    frame_files = sorted([os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg'))])
    video_frames = [cv2.imread(frame) for frame in frame_files if cv2.imread(frame) is not None]
    if len(video_frames) == 0:
        return None  # Handle the case where no frames are loaded
    video_frames = np.stack(video_frames, axis=0)
    return video_frames


def perform_3d_wavelet_transform(video_frames):
    # This is a placeholder example. You'll need to adjust it to perform an actual 3D transform.
    # For demonstration, let's just do a simple 2D transform on the first frame.
    coeffs2d = pywt.dwt2(video_frames[0], 'haar')
    # coeffs2d is a tuple: (cA, (cH, cV, cD))
    cA, (cH, cV, cD) = coeffs2d
    # Flatten the coefficients and return
    return np.concatenate([cA.flatten(), cH.flatten(), cV.flatten(), cD.flatten()])


def prepare_feature_vector(coeffs):
    # Assuming coeffs is already flattened, as from the example 3D transform function
    # Convert to a sparse matrix (COO format)
    sparse_vector = sparse.coo_matrix(coeffs)
    return sparse_vector


# Folder paths
folders = [
    "/Volumes/TOSHIBA_EXT/Phenotype_features_collective/Data/ExtractedFrames/N2/1.1_4_n2_6b_Set0_Pos0_Ch3_14012018_125942",
    "/Volumes/TOSHIBA_EXT/Phenotype_features_collective/Data/ExtractedFrames/CB4856/1.1_3_cb4856_oo_Set0_Pos0_Ch3_14012018_114743",
    "/Volumes/TOSHIBA_EXT/Phenotype_features_collective/Data/ExtractedFrames/da609/1.1_1_da609_Set0_Pos0_Ch5_14012018_092702"
]


In [2]:
import umap

# Load video frames from each folder
videos = [load_video_frames(folder) for folder in folders]

# Initialize an empty list to hold the feature vectors
feature_vectors = []

# Ensure the UMAP reducer is defined before you use it
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, metric='euclidean', spread=1.0)

# Process each video through the pipeline
for video in videos:
    if video is not None:  # Check that video frames were successfully loaded
        # Apply 3D wavelet transform
        coeffs = perform_3d_wavelet_transform(video)
        # Prepare the feature vector (now directly returns a sparse matrix)
        feature_vector = prepare_feature_vector(coeffs)
        # Append the feature vector (sparse matrix) to the list for UMAP processing
        feature_vectors.append(feature_vector)

# Check if there are any feature vectors to process
if feature_vectors:
    # Convert list of sparse matrices to a single sparse matrix
    sparse_feature_matrix = sparse.vstack(feature_vectors)

    # Proceed with UMAP
    embedding = reducer.fit_transform(sparse_feature_matrix)
    # Now, embedding contains the UMAP output which you can use as needed
else:
    print("No feature vectors were generated. Check the input video frames.")


/Users/emmarusconi/miniconda3/envs/collectiveNematode/lib/python3.8/site-packages/umap/umap_.py:2433: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(
/Users/emmarusconi/miniconda3/envs/collectiveNematode/lib/python3.8/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:1600: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  warnings.warn("k >= N for N * N square matrix. "


TypeError: Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.